### <center>__画最后一年年平均正压流函数bsf变化__

In [ ]:
output                 = "/home/yyq/mls/dataprocess/nowind/photo/bsf"
datapath               = "/data06/yyq/mls/licom2/nowind/exe/"
datapath_ctrl          = "/data06/yyq/mls/licom2/ctrl/exe/"

In [ ]:
f                      = addfile(datapath+"YME.nc", "r")
f_year_ctrl            = addfile(datapath_ctrl+"YME.nc", "r")

In [ ]:
bsf_year_a             = f->bsf(:,:,:)
bsf_year_ctrl_a        = f_year_ctrl->bsf(:,:,:)
printVarSummary(bsf_year_a)

bsf_year_end为最后一年年平均bsf

In [9]:
lon_size               = dimsizes(bsf_year_a(0,0,:)) - 3
time_size              = dimsizes(bsf_year_a(:,0,0)) - 1
lat_size               = dimsizes(bsf_year_a(0,:,0)) - 1
bsf_year_end           = bsf_year_a(time_size,:,0:lon_size)
bsf_year_ctrl_end      = bsf_year_ctrl_a(time_size,:,0:lon_size)
bsf_delta = bsf_year_end - bsf_year_ctrl_end
copy_VarMeta(bsf_year_end, bsf_delta)
printVarSummary(bsf_delta)

Variable: bsf_delta
Type: float
Total Size: 82800 bytes
            20700 values
Number of Dimensions: 2
Dimensions and sizes:	[lat | 115] x [lon | 180]
Coordinates: 
            lat: [90..-78]
            lon: [ 0..358]
Number Of Attributes: 5
  time :	1057
  long_name :	Barotropic Stream Function
  units :	Sverdrup
  missing_value :	1e+35
  _FillValue :	1e+35

level为特定等值线

In [10]:
xx                     = new((/time_size + 1/),float) 
xx                     = ispan(1,time_size + 1,1)
bsf = bsf_year_a(:,:,0:lon_size)
bsf_ctrl = bsf_year_ctrl_a(:,:,0:lon_size)
bsf_diff = bsf - bsf_ctrl
field_bsf = bsf(0,:,:)
tval_bsf = bsf(0,:,:)
do i =0,lat_size
    do j =0,lon_size
        if (.not. ismissing(tval_bsf(i,j))) then
        field_bsf(i,j) = regline(xx,bsf_diff(:,i,j))
        tval_bsf(i,j) = field_bsf@tval
        end if
    end do
end do

In [11]:
level                  = new((/81/),float)  
level                  = ispan(-80,80,2)
level                  = level / 1

Draw

In [12]:
wks      = gsn_open_wks("png", output)
plots    = new(1,"graphic")
res_t = True
    res_t@gsnFrame         = False
    res_t@gsnDraw          = False
    res_t@cnLinesOn        = False
    res_t@cnLineLabelsOn   = False
    res_t@cnFillOn         = False
    res_t@cnInfoLabelOn    = False
    res_t@gsnLeftString = ""
    res_t@gsnRightString = ""
    res_t@cnFillOn             = True             ; fill contour intervals
    res_t@cnLevelSelectionMode = "ExplicitLevels"
    res_t@lbLabelBarOn         = False
    res_t@cnMonoFillPattern    = False            ; want multiple patterns 
    res_t@cnFillPatterns       = (/17,-1,17/)     ; the patterns
    res_t@cnMonoFillScale      = False            ; want different densities 
    res_t@cnFillScales         = (/1.,1,1./)      ; the densities
    res_t@cnFillColors         = (/1,3,1/)
    res_t@cnFillDotSizeF       = 0.0025            ; increase dot size of contours
    tvall=cdft_t(0.005,28)
    res_t@cnLevels             = (/tvall,-tvall/) ;

res_0    = True
    res_0@gsnDraw                   =  False
    res_0@gsnFrame                  =  False
    res_0@cnFillOn                  =  True
    res_0@cnInfoLabelOn             =  False
    res_0@cnLinesOn                 =  False
    res_0@cnLineLabelsOn            =  False
    res_0@lbLabelBarOn              =  False
    res_0@mpCenterLonF              = 180
    res_0@cnLevelSelectionMode      = "ExplicitLevels"
    res_0@cnLevels                  = level
    res_0@lbLabelBarOn              = False
    res_0@gsnLeftString             = "delta Barotropic Stream Function"
    res_0@gsnLeftStringFontHeightF  = 15
    res_0@gsnRightString            = "Sverdrup"
    res_0@gsnRightStringFontHeightF = 15
;   res_0@cnFillPalette             = "temp_diff_18lev"
plots(0) = gsn_csm_contour_map(wks,bsf_delta, res_0)
plot_test_bsf  = gsn_csm_contour(wks,tval_bsf,res_t)
overlay(plots(0), plot_test_bsf)
resP     = True
    resP@gsnPanelLabelBar           = True
    resP@lbLabelStride              = 10
    resP@lbLabelAutoStride          = False
    resP@lbBoxSeparatorLinesOn      = False
    resP@lbBoxLinesOn               = False
    resP@lbBoxEndCapStyle           = "TriangleBothEnds"
    resP@pmLabelBarHeightF          = 0.035
    resP@pmLabelBarWidthF           = 0.9
    resP@pmLabelBarOrthogonalPosF   = 0.00
    resP@pmLabelBarParallelPosF     = 0.025
gsn_panel(wks, plots, (/1,1/), resP)

![bsf](../photo/bsf.png)